In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
print(sys.path)

['/home/d_wang/nlp/MAF', '/home/d_wang/nlp/MAF/src/baselines', '/home/d_wang/anaconda3/envs/maf/lib/python310.zip', '/home/d_wang/anaconda3/envs/maf/lib/python3.10', '/home/d_wang/anaconda3/envs/maf/lib/python3.10/lib-dynload', '', '/home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages', '../../']


In [4]:
import asyncio
import math
import os
import time
from typing import List
import pandas as pd
import torch
from tqdm import tqdm
from src.utils import Prompt, acall_gpt, call_gpt, VICUNA_MODEL_PATH, ALPACA_MODEL_PATH, get_gpu_memory
from fastchat.model.model_adapter import get_conversation_template
from fastchat.serve.inference import load_model

from src.baselines.baseline_utils import run_baseline, create_prompt_template, OSModel
from types import SimpleNamespace
import json


/home/d_wang/anaconda3/envs/maf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
args = {
    'data_dir': 'data/gsm_data',
    'save_dir': 'src/baselines/models',
    'debug': False,
    'exp_label': 'default',
    'task': 'pot_gsm',
    'model': 'vicuna',
    'max_tokens': 2048,
    'temperature': 0.0,
}
args['ckpt_path'] = os.path.join(args['save_dir'], args['exp_label'])
args = SimpleNamespace(**args)

In [6]:
vicuna = OSModel(
        engine=args.model,
        temperature=0.001,
        cuda_visible_devices="0, 1, 2"
)

0, 1, 2


Loading checkpoint shards: 100%|██████████| 3/3 [00:28<00:00,  9.36s/it]


In [7]:
await vicuna(['3 + 17'])

  0%|          | 0/1 [00:00<?, ?it/s]

len input_ids 1
len input_ids[0] 9


100%|██████████| 1/1 [01:39<00:00, 99.57s/it]


['17 + 17\n\n17 + 17 + 1\n\n17 + 17 + 1 + 1\n\n17 + 17 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1 + 1\n\n17 + 17 + 1 + 1 + 

In [8]:
for prompt_technique in ['0cot_gsm', '4cot_gsm', 'pot_gsm', 'ltm_gsm']:
    await run_baseline(vicuna, 'gsm_baseline', prompt_technique, 'vicuna', args.data_dir, args.save_dir)


  0%|          | 0/5 [00:00<?, ?it/s]

len input_ids 1
len input_ids[0] 942


 20%|██        | 1/5 [03:55<15:40, 235.16s/it]

len input_ids 1
len input_ids[0] 949


 40%|████      | 2/5 [07:51<11:47, 235.75s/it]

...

len input_ids 1
len input_ids[0] 955


 20%|██        | 1/5 [03:57<15:48, 237.13s/it]

len input_ids 1
len input_ids[0] 975


 40%|████      | 2/5 [07:57<11:56, 238.85s/it]

len input_ids 1
len input_ids[0] 964
